Install and import necesseary libraries, frameworks 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt 

2025-05-07 21:28:38.959642: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-07 21:28:39.888005: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-07 21:28:40.472122: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746642521.588164    1488 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746642521.798804    1488 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746642523.854709    1488 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Load train and test data as pandas DataFrames

In [2]:
train_df = pd.read_csv('train.csv',na_filter=False)
test_df = pd.read_csv('test.csv',na_filter=False)

Drop 'id' columns because it does not provide meaningful data to predict the calorie expenditure. Its being will cause faults when traning and testing the NN.

In [3]:
train_df.drop(columns='id',inplace=True)
test_df.drop(columns='id',inplace=True)

Processing data for model training looking for NaN values on the dataset and handling them, dropping unnecesseary columns, transform data and split data to test and train splits

In [34]:
def process_data(dataframe):
    if dataframe.isna().sum().sum() > 0 or dataframe.isnull().sum().sum() > 0:
        dataframe.dropna
    
    X = dataframe.drop(columns='Calories')
    Y = dataframe['Calories']
    
    numeric_features = ['Age','Height','Weight','Heart_Rate','Body_Temp']
    categorical_features = ['Sex']
    
    num_encoder = StandardScaler()
    cat_encoder = OneHotEncoder(drop="first", sparse_output=False)
    
    preprocessor = ColumnTransformer(transformers= [('num', num_encoder, numeric_features),('cat',cat_encoder,categorical_features)])
    
    X_precessed = preprocessor.fit_transform(X)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_precessed, Y , test_size=0.25, random_state=42)
    
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    
    return X_train, X_test, Y_train, Y_test
    

In [35]:
X_train, X_test, Y_train, Y_test = process_data(train_df)

X_train shape: (562500, 6)
X_test shape: (187500, 6)


Building Neural Network for regression traning using ReLu for activation function, optimized with adam optimizer and used a MSE for the loss function 

In [39]:
def create_model(input_shape):
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(input_shape,)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), 
                  loss='mean_squared_error', 
                  metrics=[keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None), 
                  keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)
                  ])
    
    model.summary()
    
    return model

In [40]:
model = create_model(X_train.shape[1])

/home/albatrosfirst/calorieexp_NN/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
E0000 00:00:1746617068.633150    1689 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1746617068.850269    1689 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,265 (12.75 KB)

 Non-trainable params: 192 (768.00 B)

Defining train_model function this function accepts model and traning and test data splits, creates a callback object of keras EarlyStopping class restore to the best paramaters on while traning to acces the best trarning epochs. Configuring model trainig with train data splits, batch sizes and validation data and returning model and traning history of the epochs results.

In [45]:
def train_model(model, x_train, x_test, y_train, y_test):
    callback = keras.callbacks.EarlyStopping(
        monitor='loss', 
        patience=5, 
        restore_best_weights=True
        )
    
    traning = model.fit(x= x_train,
              y= y_train,
              epochs= 100,
              callbacks= [callback],
              batch_size= 64,
              validation_data= (x_test, y_test),
              )
    
    return model, traning

In [47]:
trained_model,training = train_model(model, X_train, X_test, Y_train, Y_test)

Epoch 1/100
8790/8790 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - loss: 217.6876 - mean_absolute_error: 10.7235 - mean_squared_error: 217.6876 - val_loss: 181.3868 - val_mean_absolute_error: 9.7507 - val_mean_squared_error: 181.3868
Epoch 2/100
8790/8790 ━━━━━━━━━━━━━━━━━━━━ 86s 10ms/step - loss: 215.7873 - mean_absolute_error: 10.6510 - mean_squared_error: 215.7873 - val_loss: 182.0086 - val_mean_absolute_error: 9.8358 - val_mean_squared_error: 182.0086
Epoch 3/100
8790/8790 ━━━━━━━━━━━━━━━━━━━━ 90s 10ms/step - loss: 214.4155 - mean_absolute_error: 10.6327 - mean_squared_error: 214.4155 - val_loss: 182.7892 - val_mean_absolute_error: 9.8464 - val_mean_squared_error: 182.7892
Epoch 4/100
8790/8790 ━━━━━━━━━━━━━━━━━━━━ 120s 14ms/step - loss: 212.2582 - mean_absolute_error: 10.5865 - mean_squared_error: 212.2582 - val_loss: 182.5332 - val_mean_absolute_error: 9.8672 - val_mean_squared_error: 182.5332
Epoch 5/100
8790/8790 ━━━━━━━━━━━━━━━━━━━━ 144s 16ms/step - loss: 210.3065 - mean_absolute_error

Defening and measuring the model outputs according to test splits to measure how well the model is trained we are measuring the Mean Squared Error between predictions and the real outputs, Mean Absolute Error, Root of the MSE and lastly the R2 score to evaluate the overall quality of the model.

In [ ]:
def evaluate_model(model, x_test, y_test):
    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test,y_pred)
    rmse = np.sqrt(mse)
    
    print("Model evaluations:\n")
    print(f"Mean Squared Error: {mse}\n")
    print(f"Mean Absolute Error: {mae}\n")
    print(f"R2 Score: {r2}\n")
    print(f"Root Mean Squared Error: {rmse}\n")
    
    return mse,mae,r2,rmse

In [52]:
evaluate_model(model, X_test, Y_test)

5860/5860 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step
Model evaluations:

Mean Squared Error: 197.32586705354754

Mean Absolute Error: 10.626487284851073

R2 Score: 0.9491136793202063

Root Mean Squared Error: 14.047272584154816



Above results indicate a very well output score of %94.9 R2 score this means model explains the calorie expenditure with %95 of variance its a very good score for the first and raw traning we have done, RMSE is 14.05 this indicates model predicts calories larger than the true expenditure and lastly MAE is 10.62 directly indicates the absolute difference avg between prediction to truth. 
Now lets try to upgrade this model to a more advanced version for reducing the error scores.

**Feature Engineering**
On the above cells we use the data as given we did not create any meaninful columns combining what we have on the data we can populate the columns with what data we have examples:

***1.BMI(weight/height^2)***
Measures a persons Body Mass Index indicates if a personis underweight, normal, overweight or obese. This measurement can give us a deeper understanding for calorie predictions.

***2.Calorie Rate(calorie/duration)***
Colorie expenditure per duration.

***3.Temp adjusted heart rate***
High body temp can increase hearth rate this can indicate more calorie expenditure than nromal parameters normalizing this to 37 degrees can give us reduced calori exp.

In [4]:
def process_data_with_features(dataframe):
    if dataframe.isna().sum().sum() > 0 or dataframe.isnull().sum().sum() > 0:
        dataframe.dropna
    
    dataframe['BMI'] = dataframe['Weight'] / ((dataframe['Height']/100) ** 2)
    dataframe['Calorie_Rate'] = dataframe['Calories'] / dataframe['Duration']
    dataframe['Adjusted_HR'] = dataframe['Heart_Rate'] * (37/dataframe['Body_Temp'])
    
    X = dataframe.drop(columns='Calories')
    Y = dataframe['Calories']
    
    numeric_features = ['Age','Height','Weight','Heart_Rate','Body_Temp','BMI','Calorie_Rate','Adjusted_HR']
    categorical_features = ['Sex']
    
    num_encoder = StandardScaler()
    cat_encoder = OneHotEncoder(drop="first", sparse_output=False)
    
    preprocessor = ColumnTransformer(transformers= [('num', num_encoder, numeric_features),('cat',cat_encoder,categorical_features)])
    
    X_precessed = preprocessor.fit_transform(X)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X_precessed, Y , test_size=0.25, random_state=42)
    
    print(f"X_train_enh shape: {X_train.shape}")
    print(f"X_test_enh shape: {X_test.shape}")
    
    return X_train, X_test, Y_train, Y_test
    

In [5]:
X_train_enh, X_test_enh, Y_train_enh, Y_test_enh = process_data_with_features(train_df)

X_train_enh shape: (562500, 9)
X_test_enh shape: (187500, 9)


***Enhance Model Architecture***
Compiling a more deep layered Neural Network with deeper layers and with different loss function to increase prediction accuracy. This enhanced model uses Huber loss function that combines MSE and MAE loss function.

In [6]:
def advanced_model_building(input_shape):
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(96, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), 
                  loss= keras.losses.Huber(delta=1.0), 
                  metrics=[keras.metrics.MeanSquaredError(name="mean_squared_error", dtype=None), 
                  keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None)
                  ])
    
    model.summary()
    
    return model

In [ ]:
enhanced_model = advanced_model_building(X_train_enh.shape[1])

In [26]:
def train_enhanced_model(model, x_train, x_test, y_train, y_test):
    callbacks = [keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
    ),
    keras.callbacks.ModelCheckpoint(
    'best_param_model.h5',
    monitor='val_loss',
    save_best_only=True,    
    ),
    keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patiance=5,
    min_lr=1e-6,
    )]
    
    traning = model.fit(x= x_train,
              y=y_train,
              epochs= 150,
              callbacks=[callbacks],
              batch_size=32,
              validation_data=(x_test, y_test),
              verbose=1
              )
    
    return best_model, traning

In [ ]:
trained_enhanced_model, enhanced_model_traning = train_enhanced_model(enhanced_model, X_train_enh, X_test_enh, Y_train_enh, Y_test_enh)

Epoch 1/150
17577/17579 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 17.2944 - mean_absolute_error: 17.7833 - mean_squared_error: 1058.5295

17579/17579 ━━━━━━━━━━━━━━━━━━━━ 251s 14ms/step - loss: 17.2934 - mean_absolute_error: 17.7823 - mean_squared_error: 1058.4164 - val_loss: 8.6411 - val_mean_absolute_error: 9.1213 - val_mean_squared_error: 199.5453 - learning_rate: 0.0010
Epoch 2/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 230s 13ms/step - loss: 9.6030 - mean_absolute_error: 10.0875 - mean_squared_error: 203.7120 - val_loss: 9.0777 - val_mean_absolute_error: 9.5587 - val_mean_squared_error: 193.2251 - learning_rate: 0.0010
Epoch 3/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 238s 14ms/step - loss: 9.3795 - mean_absolute_error: 9.8629 - mean_squared_error: 195.3717 - val_loss: 8.9882 - val_mean_absolute_error: 9.4680 - val_mean_squared_error: 193.9948 - learning_rate: 0.0010
Epoch 4/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 253s 14ms/step - loss: 9.2521 - mean_absolute_error: 9.7352 - mean_squared_error: 191.1024 - val_loss: 8.9307 - val_mean_absolute_error: 9.4117 - val_mean_squared_error: 201.0615 - learning_rate: 0.0010
Epoch 5/150
1757

17579/17579 ━━━━━━━━━━━━━━━━━━━━ 230s 13ms/step - loss: 9.1024 - mean_absolute_error: 9.5851 - mean_squared_error: 186.1299 - val_loss: 8.5658 - val_mean_absolute_error: 9.0474 - val_mean_squared_error: 191.7610 - learning_rate: 0.0010
Epoch 8/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 230s 13ms/step - loss: 9.0556 - mean_absolute_error: 9.5382 - mean_squared_error: 184.0773 - val_loss: 8.7339 - val_mean_absolute_error: 9.2153 - val_mean_squared_error: 192.1521 - learning_rate: 0.0010
Epoch 9/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 228s 13ms/step - loss: 9.0334 - mean_absolute_error: 9.5157 - mean_squared_error: 183.3579 - val_loss: 8.7134 - val_mean_absolute_error: 9.1950 - val_mean_squared_error: 187.0565 - learning_rate: 0.0010
Epoch 10/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 231s 13ms/step - loss: 9.0401 - mean_absolute_error: 9.5224 - mean_squared_error: 184.3229 - val_loss: 8.6933 - val_mean_absolute_error: 9.1738 - val_mean_squared_error: 195.9092 - learning_rate: 0.0010
Epoch 11/150
17579/

17579/17579 ━━━━━━━━━━━━━━━━━━━━ 252s 14ms/step - loss: 8.9381 - mean_absolute_error: 9.4199 - mean_squared_error: 180.9547 - val_loss: 8.5617 - val_mean_absolute_error: 9.0410 - val_mean_squared_error: 185.1270 - learning_rate: 0.0010
Epoch 15/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 260s 15ms/step - loss: 8.9474 - mean_absolute_error: 9.4292 - mean_squared_error: 180.4247 - val_loss: 8.7259 - val_mean_absolute_error: 9.2055 - val_mean_squared_error: 176.0440 - learning_rate: 0.0010
Epoch 16/150
17579/17579 ━━━━━━━━━━━━━━━━━━━━ 265s 15ms/step - loss: 8.9522 - mean_absolute_error: 9.4338 - mean_squared_error: 181.1817 - val_loss: 8.8641 - val_mean_absolute_error: 9.3449 - val_mean_squared_error: 184.2149 - learning_rate: 0.0010
Epoch 17/150
17577/17579 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8.9272 - mean_absolute_error: 9.4092 - mean_squared_error: 180.0956

In [5]:
def plot_traning_session(traning_history):
    plt.Figure(figsize=(12,12))
    
    plt.subplot(1,2,1)
    plt.plot(traning_history.history['loss'])
    plt.plot(traning_history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    
    plt.show()